In [1]:
import pickle
import spell
import emoji
import string
import re

In [2]:
from enchant.checker import SpellChecker
from enchant.tokenize import EmailFilter, URLFilter
import enchant
import string

import nltk

import ftfy
from collections import Counter

In [3]:
from cleaning_refs import med_dur_conversion as med_dur_conversion
from cleaning_refs import cList as cList
from cleaning_refs import new_words as new_words

In [4]:
def load_pickle(_pickle_name):
    return pickle.load( open( _pickle_name, "rb" ) )

In [5]:
# c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text):#, c_re=c_re):
    c_re = re.compile('(%s)' % '|'.join(cList.keys()))
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [6]:
def extract_emojis(str):
    return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)

In [7]:
#method of review?
def metaData_clean(_reviews):
    for _review in _reviews:
        # fix ageRange to interval
        try:
            temp_ar = _review.ageRange.split('-')
            _review.ageRange = [int(temp_ar[0]), int(temp_ar[1])]
        except:
            if type(_review.ageRange) != type([]):
#                 print('age range', _review['ageRange'])
                continue

        # fix medDuration to interval of months **[, ) interval
        try:
#             _review.medDuration_int']= str(med_dur_conversion[_review['medDuration']])
            _review.medDuration= med_dur_conversion[_review.medDuration]
        except:
            if type(_review.medDuration) != type([]):
#                 print('wait, what?', _review['medDuration'])
                continue

        for value in [_review.effectiveness, _review.ease_of_use, _review.satisfaction, _review.genRating, _review.upVotes]:
            try:
                _review[value] = int(_review[value])
            except:
                continue  

                
        #pull out contractions
        comment_text = (_review.comment).split()
        for word in comment_text:
#             print(extract_emojis(word))
            word = expandContractions(word, c_re=c_re)
        _review.comment = ' '.join(comment_text)
        
        #remove punctuation
        remove = string.punctuation
        remove = remove.replace("-", "") # don't remove hyphens
        pattern = r"[{}]".format(remove) 
        _review.comment = re.sub(pattern, "",  _review.comment) 
        
    return _reviews

In [8]:
d2 = enchant.DictWithPWL("en_US","new_words.txt")
chkr = SpellChecker("en_US", filters=[EmailFilter,URLFilter])

def spellchecker(reviews):
    for ik, review in enumerate(reviews):
    #         print ('-------------')
    #         print (review['comment'])
    #         print(ftfy.fix_text(review['comment'])#['review'])
    #         words = [word.lower() for word in words]
            chkr.set_text(review.comment)
            for err in chkr:
                error = (err.word).lower()
                options = [change.lower() for change in d2.suggest(error)]
                if error in options:
                    continue
                else:
                    catch = 'no'
                    word_to_beat = options[0]
                    score_to_beat = 10
                    for word in options:
                        dist = nltk.edit_distance(error, word)
                        if (word in new_words):# and (nltk.edit_distance(error, word)<score_to_beat):
                            if (nltk.edit_distance(error, word)<=score_to_beat):
                                word_to_beat = word
                                score_to_beat = nltk.edit_distance(error, word)
                                catch = 'yes'
                        elif dist<score_to_beat:
                            word_to_beat = word
                            score_to_beat = nltk.edit_distance(error, word)
                    print('final option:', error, word_to_beat, score_to_beat)

In [14]:
pickle_name = "drug_list_ddc.p"

In [15]:
from drugSite_scrapers import drug, review
drug_list = load_pickle(pickle_name)
print(pickle_name)

In [11]:
drug_list[1].__dict__

{'generic': 'Escitalopram Oxalate',
 'name': 'Lexapro',
 'num_rev': defaultdict(int, {'wmd': 4088}),
 'num_rev_pages': defaultdict(int, {'wmd': 818}),
 'reviews': defaultdict(list,
             {'wmd': [<drugSite_scrapers.review at 0x782b370>,
               <drugSite_scrapers.review at 0x782b5d0>]}),
 'score': defaultdict(int, {}),
 'url_drug': defaultdict(str,
             {'wmd': 'http://www.webmd.com/drugs/2/drug-63990-2095/lexapro/details'}),
 'url_drug_revs': defaultdict(str,
             {'wmd': 'http://www.webmd.com/drugs/drugreview-63990-lexapro.aspx?drugid=63990&amp;drugname=lexapro'})}

In [16]:
len(drug_list[0].reviews['ddc'])#[0].comment

25

In [ ]:
print(pickle_name)
for drug in drug_list:
    reviews = drug
reviews = metaData_clean(reviews)